# Login and Initialize
## Login Using Spotify
After running this click Sign-In and login to Spotify. **Note** you cannot run the whole Notebook through. Run cell #1 first.

In [ ]:
%load_ext autoreload
%autoreload 2
from ipyauth import ParamsSpotify, Auth
auth = Auth(ParamsSpotify(
    redirect_uri='http://localhost:8888/callback', 
    client_id="9e4657eefbac41afa98c61f590d8fd51"))
auth

## Fetch the Library
Initialize the Spotify object with a `logger` and fetch the library DataFrame. Check it works by outputting the Profile image.

In [ ]:
import requests
from IPython.display import Image
from spotify import Spotify
from pandas import DataFrame as df
from datetime import datetime,timedelta,date
import json
import logging

logging.basicConfig()
defaultLogger = logging.getLogger()
defaultLogger.setLevel(logging.ERROR)
logger = logging.getLogger('SpotifyLab')
logger.setLevel(logging.DEBUG)

spot = Spotify(auth, logger = logger)
lib = spot.fetchLibraryDataFrame( cache=False)
# Common Clauses
released1Y = (lib.track_released>(datetime.now()-timedelta(days=365)))
released4M = (lib.track_released>(datetime.now()-timedelta(days=120)))
released3M = (lib.track_released>(datetime.now()-timedelta(days=90)))
added2W = (lib.added_at>(datetime.now()-timedelta(days=14)).isoformat())
added3M = (lib.added_at>(datetime.now()-timedelta(days=90)).isoformat())
added4M = (lib.added_at>(datetime.now()-timedelta(days=120)).isoformat())

#lib.columns.values
Image(url=spot.imageUrl, width=50)

## Holiday Playlists



In [ ]:
decSongs = spot.fetchPlaylistUris("December Songs")
holidaySongs = spot.fetchPlaylistUris("Holiday Songs")
allDecember = lib[(lib.original_uri.isin(decSongs)) | (lib.original_uri.isin(holidaySongs)) ]
#lib.columns.values
import pandas as pd

#allDecember.head(1)
lib[pd.isnull(lib.danceability)][["track_name","artist","original_uri","name","original_id"]]

spot.updatePlaylist("Auto December",allDecember)
spot.updatePlaylist("Auto Christmas Party", allDecember[ (allDecember.energy>0.7) & (allDecember.acousticness<0.5) ])
spot.updatePlaylist("Auto Holly Jolly", allDecember[ (allDecember.energy>0.6) & (allDecember.instrumentalness<0.9) & (allDecember.acousticness<0.7) ])
spot.updatePlaylist("Auto December Chill", allDecember[ (allDecember.energy<0.701) & (allDecember.instrumentalness>0.2) ])
spot.updatePlaylist("Auto Silent Night", allDecember[ (allDecember.danceability<0.4) & (allDecember.energy<0.4) & (allDecember.instrumentalness>0.4) ])
spot.updatePlaylist("Auto Work Christmas", allDecember[ (allDecember.instrumentalness>0.2) & (allDecember.energy>0.2) ])
spot.updatePlaylist("Auto Instrumental Christmas", allDecember[ allDecember.instrumentalness>0.75 ])
spot.updatePlaylist("Auto Added Christmas",allDecember[ allDecember.added_at>(datetime.now()-timedelta(days=7)).isoformat()])
spot.updatePlaylist("Auto Recent Christmas",allDecember[ allDecember.track_released>(datetime.now()-timedelta(days=365))])
spot.updatePlaylist("Auto Classic Christmas",allDecember[ allDecember.track_released<(datetime(1991,1,1))])

#lib.columns.values


# Playlist Updates

## A Running Playlist

This playlist has high tempo and energy. The preference is for instrumentals with high danceability. Avoid long songs to keep the tracks changing.

In [ ]:
newPlaylist = lib[
    (lib.tempo>130) & 
    (lib.instrumentalness>0.3) & 
    (lib.duration_ms<1000*60*60) & 
    (lib.danceability>0.7) &
    (lib.track_released>datetime(1990,1,1))
].sort_values(["added_at"])
# Exclude some artists from this playlist
newPlaylist = newPlaylist[~newPlaylist.artist.isin(["Radiohead","Rodrigo y Gabriela","2CELLOS","Faithless","Doves"])]
spot.updatePlaylist("Auto Run Fast",newPlaylist,excludingPlaylists=["Holiday Songs"])

## Unpopular or Hispter?
This playlists tracks tracks with low popularity. This is useful for checking if you added the most commonly played track (e.g. a version before it was remastered).

In [ ]:
spot.updatePlaylist("Are You Sure",lib[lib.track_popularity<2],excludingPlaylists=["Holiday Songs","Noise"])

## Decades of Music
These playlists bucket the music in your library into decades using the release date. 

In [ ]:
spot.updatePlaylist("Auto 1960s",lib[lib.track_released<datetime(1970,1,1)],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Auto 1970s",lib[(lib.track_released>datetime(1970,1,1)) & (lib.track_released<datetime(1980,1,1))],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Auto 1980s",lib[(lib.track_released>datetime(1980,1,1)) & (lib.track_released<datetime(1990,1,1))],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Auto 1990s",lib[(lib.track_released>datetime(1990,1,1)) & (lib.track_released<datetime(2000,1,1))],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Auto 2000s",lib[(lib.track_released>datetime(2000,1,1)) & (lib.track_released<datetime(2010,1,1))],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Auto 2010s",lib[(lib.track_released>datetime(2010,1,1)) & (lib.track_released<datetime(2020,1,1))],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Good Year",lib[released1Y],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Auto 2020",lib[lib.track_released>=datetime(2020,1,1)],excludingPlaylists=["Holiday Songs"])


## Recently Added
The music you added in the last couple of months.

In [ ]:
spot.updatePlaylist("Auto 2W",lib[added2W],excludingPlaylists=["Holiday Songs","Noise"])
spot.updatePlaylist("Auto 3M",lib[added3M],excludingPlaylists=["Holiday Songs","Noise","Auto 2W"])
spot.updatePlaylist("New Released",lib[released3M],excludingPlaylists=["Holiday Songs","Noise","Auto 2W"])
spot.updatePlaylist("New Mix Takes",lib[released3M & (lib.energy>0.3)],excludingPlaylists=["Holiday Songs","Auto 2W"])

## Mood Music
Music for every occasion. Low energy dinner parties, going to sleep or concentrating at work.

In [ ]:
spot.updatePlaylist("Dinner Party",lib[(lib.acousticness>0.9) & (lib.energy<0.6)],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Night Nights",lib[(lib.acousticness>0.9) & (lib.energy<0.3) & (lib.danceability<0.3)],excludingPlaylists=["Holiday Songs"])

workInstrumental = (lib.instrumentalness>0.88)
workDance = (lib.danceability>0.5)
quietInstrumental = (lib.instrumentalness>0.90)
quietDance = (lib.danceability<0.6)
quietEnergy = (lib.loudness<-15)

spot.updatePlaylist("To Work To",lib[workInstrumental & workDance ],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("To Work To 4M",lib[workInstrumental & workDance & added4M],excludingPlaylists=["Holiday Songs","Noise"])
spot.updatePlaylist("New Released To Work To",lib[workInstrumental & workDance & released1Y],excludingPlaylists=["Holiday Songs"])

# With new Music Only
spot.updatePlaylist("Quiet Work 4M",lib[quietInstrumental & quietDance & quietEnergy & added4M],excludingPlaylists=["Holiday Songs","Noise"])
spot.updatePlaylist("New Released Quiet Work",lib[quietInstrumental & quietDance & quietEnergy & released1Y],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Quiet Work",lib[quietInstrumental & quietDance & quietEnergy ],excludingPlaylists=["Holiday Songs"])



## More Analysis
It's easy to add the same track twice, from different albums. This can cause duplicates in playlists. We can use Pandas `groupby` to scan for that

In [ ]:
# Duplicates in the library
ff = lib.groupby(["track_name","artist","track_duration_ms"]).filter(lambda x : len(x)>1)
spot.updatePlaylist("Duplicates",ff,excludingPlaylists=["Holiday Songs"])

#lib[[lib.track_id.isin(ff["track_id"].values)]]